# Level 7: Working with SQLite in Python (Raw SQL)

This notebook takes a deeper dive into using Python's built-in `sqlite3` module. We will focus on best practices for executing queries safely and efficiently, handling transactions, and preventing common security vulnerabilities like SQL injection.

### Setup
Let's create a fresh database for our examples.

In [1]:
import sqlite3
import os

db_file = 'python_sqlite.db'
if os.path.exists(db_file):
    os.remove(db_file)

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE
);
""")
conn.commit()

## 7.1 Using the `sqlite3` Module

The basic flow is `connect -> cursor -> execute -> fetch/commit -> close`.

### Fetching Results
After a `SELECT` query, you can retrieve the results in several ways:

In [2]:
cursor.execute("INSERT INTO users (name, email) VALUES (?, ?)", ('Alice', 'alice@example.com'))
cursor.execute("INSERT INTO users (name, email) VALUES (?, ?)", ('Bob', 'bob@example.com'))
conn.commit()

cursor.execute("SELECT * FROM users;")

# .fetchone(): Fetches the next row in the result set.
print("Fetch one:", cursor.fetchone())

# .fetchall(): Fetches all remaining rows.
cursor.execute("SELECT * FROM users;") # Re-execute query
print("Fetch all:", cursor.fetchall())

Fetch one: (1, 'Alice', 'alice@example.com')
Fetch all: [(1, 'Alice', 'alice@example.com'), (2, 'Bob', 'bob@example.com')]


## 7.2 Parameterized Queries (Prevent SQL Injection)

**SQL Injection** is a common attack where malicious SQL code is inserted into input fields. **Never** use Python's f-strings or string formatting (`%`) to build queries with user input.

In [3]:
# DANGEROUS: DO NOT DO THIS!
user_input = "'Alice'; DROP TABLE users; --"
# query = f"SELECT * FROM users WHERE name = '{user_input}'"
# print(query) # This would execute a malicious command

### The Correct Way: Use Placeholders
The `sqlite3` module will safely handle the parameters for you.

In [4]:
user_name = 'Alice'

# Use a '?' as a placeholder for each parameter
cursor.execute("SELECT * FROM users WHERE name = ?", (user_name,))
print("Safely fetched user:", cursor.fetchone())

Safely fetched user: (1, 'Alice', 'alice@example.com')


## 7.3 Transactions

A **transaction** is a sequence of operations performed as a single logical unit of work. All operations in a transaction must succeed; if any one of them fails, the entire transaction is rolled back.

- `conn.commit()`: Saves the changes made in the current transaction.
- `conn.rollback()`: Reverts any changes made since the last commit.

In [5]:
try:
    # Try to insert a user with an email that already exists (violates UNIQUE constraint)
    cursor.execute("INSERT INTO users (name, email) VALUES (?, ?)", ('Another Alice', 'alice@example.com'))
    conn.commit() # This line won't be reached
except sqlite3.IntegrityError as e:
    print(f"Error: {e}")
    print("Rolling back transaction...")
    conn.rollback()

Error: UNIQUE constraint failed: users.email
Rolling back transaction...


### Using a Context Manager
The connection object can be used as a context manager, which automatically handles committing on success or rolling back on an exception.

In [6]:
try:
    with conn:
        # This will automatically be rolled back if an error occurs inside the 'with' block
        conn.execute("INSERT INTO users (name, email) VALUES (?, ?)", ('Charlie', 'charlie@example.com'))
        conn.execute("INSERT INTO users (name, email) VALUES (?, ?)", ('Another Bob', 'bob@example.com')) # This will fail
except sqlite3.IntegrityError as e:
    print(f"\nError in context manager: {e}")

# Check if Charlie was added (he shouldn't have been, due to the rollback)
cursor.execute("SELECT * FROM users WHERE name = 'Charlie'")
print("Was Charlie added?", cursor.fetchone())


Error in context manager: UNIQUE constraint failed: users.email
Was Charlie added? None


## 7.4 Reading & Writing Data Efficiently

### `executemany()` for Bulk Inserts
As seen in a previous notebook, `executemany` is much faster for inserting multiple rows than looping with `execute`.

In [7]:
new_users = [
    ('David', 'david@example.com'),
    ('Eva', 'eva@example.com')
]

with conn:
    conn.executemany("INSERT INTO users (name, email) VALUES (?, ?)", new_users)

cursor.execute("SELECT COUNT(*) FROM users")
print(f"Total users in table now: {cursor.fetchone()[0]}")

Total users in table now: 4


In [8]:
conn.close()